In [54]:
from genericpath import isfile
import json
import os
if __name__ == '__main__':
    #os.sys.path.append('./pytorch_geometric/torch_geometric')
    os.sys.path.append('./src')
from src.model.model import MMGNet
from src.model.SGFN_MMG.baseline_sgfn import SGFN
from src.model.SGFN_MMG.baseline_sgpn import SGPN
from src.utils.config import Config
from utils import util
import torch
import argparse
import torch.nn.utils.prune as prune

In [55]:
def load_config():
    # load config file
    config = Config("./config/mmgnet.json")
    #print(config)
    config.LOADBEST = ''
    config.MODE = 'train'
    config.exp = 'test'
    config.NAME = 'sgfn'
    
    return config

In [56]:
config = load_config()
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
util.set_random_seed(config.SEED)

# if config.VERBOSE:
#     print(config)

model = MMGNet(config)
model


model name : sgfn
=== 160 classes ===
| 0             armchair:0.006|| 1             backpack:0.019|
| 2                  bag:0.005|| 3                 ball:0.143|
| 4                  bar:0.071|| 5                basin:0.286|
| 6               basket:0.009|| 7         bath cabinet:0.016|
| 8              bathtub:0.036|| 9                  bed:0.009|
|10        bedside table:0.061||11                bench:0.011|
|12                bidet:0.200||13                  bin:0.037|
|14              blanket:0.007||15               blinds:0.014|
|16                board:0.087||17                 book:0.050|
|18                books:0.080||19            bookshelf:0.065|
|20               bottle:0.041||21                  box:0.001|
|22                bread:0.286||23               bucket:0.015|
|24              cabinet:0.004||25               carpet:0.061|
|26              ceiling:0.002||27                chair:0.001|
|28             cleanser:0.500||29                clock:0.059|
|30              

NotImplementedError: 

In [41]:
model.model.gcn.gconvs[1]

GraphEdgeAttenNetwork(
  (index_get): Gen_Index()
  (index_aggr): Aggre_Index()
  (edgeatten): MultiHeadedEdgeAttention(
    (nn_edge): Sequential(
      (0): Linear(in_features=1280, out_features=768, bias=True)
      (1): ReLU()
      (2): Linear(in_features=768, out_features=256, bias=True)
    )
    (nn): mySequential(
      (0): Conv1d(96, 96, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Conv1d(96, 32, kernel_size=(1,), stride=(1,))
    )
    (proj_edge): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
    )
    (proj_query): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
    )
    (proj_value): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
    )
  )
  (prop): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): ReLU()
    (2): Linear(in_features=768, out_features=512, bias=True)
  )
)

In [ ]:
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')
    elif isinstance(module, torch.nn.Conv1d):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')

In [22]:
model.model.mmg.self_attn

AttributeError: 'SGFN' object has no attribute 'mmg'

In [12]:
model.model.mmg.self_attn[0]

GraphEdgeAttenNetwork(
  (index_get): Gen_Index()
  (index_aggr): Aggre_Index()
  (edgeatten): MultiHeadedEdgeAttention(
    (nn_edge): Sequential(
      (0): Linear(in_features=1536, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=512, bias=True)
    )
    (nn): mySequential(
      (0): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): Conv1d(128, 32, kernel_size=(1,), stride=(1,))
    )
    (proj_edge): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
    )
    (proj_query): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
    )
    (proj_value): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
    )
  )
  (prop): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): ReLU()
    (2): Linear(in_features=768, out_features=512, bias=True)
  )
)

In [11]:
model.model.mmg.self_attn[0].attention.fc_o

Linear(in_features=512, out_features=512, bias=True)

In [20]:
import torch_pruning as tp

In [29]:
head_id = 0
for m in model.model.mmg.modules():
    print("Head #%d"%head_id)
    print("[Before Pruning] Num Heads: %d, Head Dim: %d =>"%(m.num_heads, m.head_dim))
    print()
    head_id+=1

Head #0


AttributeError: 'MMG_student' object has no attribute 'head_dim'